Here I'm going to explore the notification moduel and how they work. First, I need to understand their fields

In [1]:
import pandas as pd
from pandas.io.json import json_normalize

# LOAD DOS DADOS

In [2]:
df = pd.read_json("../../data/raw_data/base_22012019.json")

In [152]:
grades_df = pd.read_excel("../../data/raw_data/nota_provas.xls")

# EXPLORAÇÃO inicial

In [3]:
notifications_df = df[df["model"] == "notifications.notification"]

In [4]:
notifications_df.shape

(13219, 3)

In [5]:
notifications_df["fields"].head(10)[34636]

{'meta': None,
 'task': 1,
 'user': 8,
 'level': 3,
 'viewed': False,
 'creation_date': '2018-10-28'}

Uma notificação possui os seguintes campos no log:
1. meta: 
2. user: que coleta o id do user para qual a notificação foi enviada
3. Level: O nível da notificação está relacionado à tarefa que ela notifica sobre, este campo possui 4 valores mas somente 3 significados
    
    3.1 1,2: Significam que a tarefa está em dia 
    
    3.2 3: Significa que a tarefa está atrasada
    
    3.3 4: Significa que o prazo para realizar a tarefa já acabou
4. viewed: que possui valor de verdadeiro ou falso indicando se a notifcação foi vista ou não pelo usuário
5. creation_date: A data de criação daquela notificação
6. task: id da task que aquela notificação está associada

# Limpeza dos dados

é preciso extrair as colunas existentes dentro da coluna "fields", para isso, eu vou utilizar a função json_normalize, nativa do pandas.

In [6]:
def extract_flatten_dataframe(df, column, meta_list):
    df_fields = json_normalize(data=df[column], meta=meta_list)
    df_fields.index = df.index
    return df_fields.join(df, how="outer")

In [184]:
notification_fields = json_normalize(data=notifications_df["fields"], meta=["meta", "task", "user", "level", "viewed", "creation_date"] )
notification_fields.index = notifications_df.index
notification_fields = notification_fields.astype({"level": pd.CategoricalDtype()})
notification_fields["creation_date"] = pd.to_datetime(notification_fields["creation_date"])
notification_df_flatted = notification_fields.join(notifications_df, how="outer")

In [185]:
notification_df_flatted.dtypes

creation_date    datetime64[ns]
level                  category
meta                     object
task                      int64
user                      int64
viewed                     bool
fields                   object
model                    object
pk                       object
dtype: object

In [182]:
notification_df_flatted.sample(10)

,creation_date,level,meta,task,user,viewed,fields,model,pk
41533,2018-12-04,3,None,2,10,False,"{'meta': None, 'task': 2, 'user': 10, 'level':...",notifications.notification,7070
44518,2019-01-15,3,None,42,51,False,"{'meta': None, 'task': 42, 'user': 51, 'level'...",notifications.notification,10165
40206,2018-11-29,3,None,11,41,False,"{'meta': None, 'task': 11, 'user': 41, 'level'...",notifications.notification,5693
46040,2019-01-18,4,None,8,53,False,"{'meta': None, 'task': 8, 'user': 53, 'level':...",notifications.notification,11687
44882,2019-01-16,1,None,45,76,False,"{'meta': None, 'task': 45, 'user': 76, 'level'...",notifications.notification,10529
47848,2019-01-22,4,None,8,63,False,"{'meta': None, 'task': 8, 'user': 63, 'level':...",notifications.notification,13495
39233,2018-11-25,1,None,17,44,True,"{'meta': None, 'task': 17, 'user': 44, 'level'...",notifications.notification,4680
37073,2018-11-18,3,None,12,41,False,"{'meta': None, 'task': 12, 'user': 41, 'level'...",notifications.notification,2450
37224,2018-11-18,3,None,1,97,False,"{'meta': None, 'task': 1, 'user': 97, 'level':...",notifications.notification,2601
35188,2018-11-07,1,None,7,78,False,"{'meta': None, 'task': 7, 'user': 78, 'level':...",notifications.notification,555


In [186]:
notification_fields.dtypes

creation_date    datetime64[ns]
level                  category
meta                     object
task                      int64
user                      int64
viewed                     bool
dtype: object

Como dá pra ver o pandas não consegue inferir muito bem os data types das colunas, vou utilizar meu conhecimento sobre elas para colocar valores mais baratos e que correspondem melhor aos seus valores.m

# Resources

In [187]:
resources_df = df[df["model"] == "topics.resource"]

In [188]:
df[df["model"] == "topics.resource"]["fields"][120546]

{'_my_subclass': 'webpage',
 'name': 'Avaliação Diagnóstica',
 'slug': 'avaliacao-diagnostica',
 'brief_description': '<div>\r\n  <small>Valor: 0,25 ponto (<font color="#ff0000"><i>extra</i></font>)</small>\r\n</div>\r\n<div>\r\n  <small>Início: 05/11/2018</small>&nbsp;<small style="margin-left: 10px">Final: 11/11/2018</small>\r\n</div>',
 'show_window': False,
 'all_students': True,
 'visible': True,
 'order': 1,
 'topic': 5,
 'create_date': '2018-10-23T03:36:54.806Z',
 'last_update': '2018-11-05T12:08:57.125Z',
 'students': [],
 'groups': [],
 'tags': [62, 63]}

In [50]:
resource_meta_list = ["_my_subclass", "visible", "students", "groups", "tags", "all_students", "order", "topic", 
             "show_window", "brief_description", "slug", "name"]

In [51]:
resources_flatten_df = extract_flatten_dataframe(resources_df, "fields", resource_meta_list).drop("fields", axis=1)
resources_flatten_df.head(5)

,_my_subclass,all_students,brief_description,create_date,groups,last_update,name,order,show_window,slug,students,tags,topic,visible,model,pk
120546,webpage,True,"<div>\r\n <small>Valor: 0,25 ponto (<font col...",2018-10-23T03:36:54.806Z,[],2018-11-05T12:08:57.125Z,Avaliação Diagnóstica,1,False,avaliacao-diagnostica,[],"[62, 63]",5,True,topics.resource,9
120547,pdffile,True,,2018-10-23T19:05:01.720Z,[],2018-12-17T15:06:52.038Z,"Prova anterior (prova 01, 2007.2, turma E6)",37,True,prova-anterior-prova-01-20072-turma-e6,[],"[10, 11, 12, 13, 14, 15, 16, 17]",6,True,topics.resource,10
120548,pdffile,True,,2018-10-23T19:06:18.936Z,[],2018-12-17T15:06:52.052Z,"Prova anterior (prova 01, 2008.1, turma 52)",38,True,prova-anterior-prova-01-20081-turma-52,[],"[10, 11, 12, 13, 14, 15, 16, 17]",6,True,topics.resource,11
120549,pdffile,True,,2018-10-23T19:07:09.839Z,[],2018-12-17T15:06:52.063Z,"Prova anterior (prova 01, 2008.2, turma A5)",39,True,prova-anterior-prova-01-20082-turma-a5,[],"[10, 11, 12, 13, 14, 15, 16, 17]",6,True,topics.resource,12
120550,pdffile,True,,2018-10-23T19:07:53.570Z,[],2018-12-17T15:06:52.078Z,"Prova anterior (prova 01, 2009.2, turma A5)",40,True,prova-anterior-prova-01-20092-turma-a5,[],"[10, 11, 12, 13, 14, 15, 16, 17]",6,True,topics.resource,13


# Pendências

In [146]:
df[df["model"] == "pendencies.pendencies"]["fields"][120708]

{'action': 'view',
 'begin_date': '2018-10-28T02:53:00Z',
 'end_date': '2018-10-28T02:55:00Z',
 'limit_date': None,
 'resource': 163}

# Users Data
Como os usuários são o principal objeto de pesquisa deste relatório, eu preciso coletar os IDs deles para juntar as notificações e "dar à um responsável".

In [147]:
users_df = df[df["model"] == "users.user"]
users_df

,fields,model,pk
34528,{'password': 'pbkdf2_sha256$30000$5DROf4Pf3BPR...,users.user,1
34529,{'password': 'pbkdf2_sha256$30000$u3Lg5la328P5...,users.user,2
34530,{'password': 'pbkdf2_sha256$30000$Tg55KDswGl6t...,users.user,3
34531,{'password': 'pbkdf2_sha256$30000$sN16VkfYDIiS...,users.user,4
34532,{'password': 'pbkdf2_sha256$30000$x85db28ZviCz...,users.user,6
34533,{'password': 'pbkdf2_sha256$30000$YuhP7bGSlXwn...,users.user,7
34534,{'password': 'pbkdf2_sha256$30000$l7WAB2nBwfTg...,users.user,8
34535,{'password': 'pbkdf2_sha256$30000$DOj6RbuvrpDK...,users.user,9
34536,{'password': 'pbkdf2_sha256$30000$wbwdFEajjHed...,users.user,10
34537,{'password': 'pbkdf2_sha256$30000$DOPLvJQYYOz4...,users.user,11


In [148]:
users_df["fields"][34528]

{'password': 'pbkdf2_sha256$30000$5DROf4Pf3BPR$tPd1JbDlLqrUzoY/JcUSfparBrp7n0ERZKWgpfnbG/Y=',
 'last_login': '2019-01-22T12:36:36.447Z',
 'is_superuser': True,
 'email': 'admin@amadeus.br',
 'username': 'Administrador',
 'last_name': 'Geral',
 'social_name': None,
 'description': '',
 'image': '',
 'date_created': '2018-10-19T16:55:27.084Z',
 'last_update': '2018-11-04T03:31:13.036Z',
 'show_email': 1,
 'is_staff': True,
 'is_active': True,
 'groups': [],
 'user_permissions': []}

# Dicionário dos dados
1. Password: Senha criptografada
2. Last_Login: a última vez que o usuário logou no sistema
3. is_superuser: Significa que o usuário é um super usuário, tem privilégios ou admin.
4. email: e-mail pertecente ao usuário 
5. username: nome que o usuário quer que outros usuários o vejam
6. last_name: Sobrenome do usuário
7. social_name: Nome social que o usuário quer escolher para ser demonstrado 
8. description: ?
9. image: Caminho para a imagem do usuário
10. date_created: Data em que o usuário foi criado
11. last_update: a última vez que os dados do usuário foram modificados
12. show_email: Uma booleana que informa se o e-mail é visiível para outros usuários
13. is_staff: se ele é do tipo admin
14. is_active: Se o usuário está ativo, caso sim, ele pode entrar no sistema, senão, é impossível
15. groups: Grupos de permissão ao qual esse usuário pertence ("professor", "estudante"...)
16. user_permissions = permissões individuais que ele possui ("criar tópico" , "deletar tópico", "editar tópico")

In [149]:
# fields:  
user_fields = ["password", "last_login", "is_superuser", "email", "username", "last_name", "social_name", "description", 
               "imagem", "date_created", "last_update", "show_email", "is_staff", "is_active", "groups", "user_permissions"]

In [150]:
user_df_flatten = extract_flatten_dataframe(users_df, column="fields",meta_list=user_fields).astype({"pk": "int64"})

In [151]:
user_df_flatten.dtypes

date_created        object
description         object
email               object
groups              object
image               object
is_active             bool
is_staff              bool
is_superuser          bool
last_login          object
last_name           object
last_update         object
password            object
show_email           int64
social_name         object
user_permissions    object
username            object
fields              object
model               object
pk                   int64
dtype: object

# Notas dos alunos

In [153]:
grades_df.dtypes

id_estudante      int64
v1              float64
v2              float64
dtype: object

In [154]:
grades_df.head(10)

,id_estudante,v1,v2
0,11,0.0,0.00
1,12,4.0,7.00
2,13,6.5,7.50
3,14,6.5,10.00
4,15,6.0,8.50
5,16,6.0,3.50
6,17,0.5,4.00
7,18,1.5,7.25
8,19,7.5,8.50
9,20,2.0,9.75


# Vou remover as seguintes colunas:
1. Groups, pois não possui nenhum valor diferente de vazio (ou nulo).

In [155]:
user_df_flatten["groups"].value_counts()

[]    106
Name: groups, dtype: int64

In [156]:
user_df_flatten_clean = user_df_flatten.drop(["groups", "fields", "password"], axis=1)

# Junção entre os datasets
Para otimizar, irei coletar somente o username dos usuários durante o join com as notificações, depois eu irei buscar mais informações sobre eles.m

In [189]:
notifications_with_username_df = pd.merge(notification_df_flatted, user_df_flatten_clean.loc[:, ["pk"]], left_on=["user"], right_on=["pk"], suffixes=("_notifications", "_user"))

In [190]:
notifications_with_user_grades_df = pd.merge(notifications_with_username_df, grades_df, left_on=["user"], right_on=["id_estudante"])

In [191]:
notifications_with_user_grades_df.head(10)

,creation_date,level,meta,task,user,viewed,fields,model,pk_notifications,pk_user,id_estudante,v1,v2
0,2018-11-05,1,None,9,11,False,"{'meta': None, 'task': 9, 'user': 11, 'level':...",notifications.notification,105,11,11,0.0,0.0
1,2018-11-06,1,None,9,11,False,"{'meta': None, 'task': 9, 'user': 11, 'level':...",notifications.notification,195,11,11,0.0,0.0
2,2018-11-06,1,None,10,11,False,"{'meta': None, 'task': 10, 'user': 11, 'level'...",notifications.notification,261,11,11,0.0,0.0
3,2018-11-06,1,None,7,11,False,"{'meta': None, 'task': 7, 'user': 11, 'level':...",notifications.notification,338,11,11,0.0,0.0
4,2018-11-07,1,None,10,11,False,"{'meta': None, 'task': 10, 'user': 11, 'level'...",notifications.notification,463,11,11,0.0,0.0
5,2018-11-07,1,None,8,11,False,"{'meta': None, 'task': 8, 'user': 11, 'level':...",notifications.notification,574,11,11,0.0,0.0
6,2018-11-08,1,None,10,11,False,"{'meta': None, 'task': 10, 'user': 11, 'level'...",notifications.notification,682,11,11,0.0,0.0
7,2018-11-08,1,None,11,11,False,"{'meta': None, 'task': 11, 'user': 11, 'level'...",notifications.notification,756,11,11,0.0,0.0
8,2018-11-08,1,None,8,11,False,"{'meta': None, 'task': 8, 'user': 11, 'level':...",notifications.notification,844,11,11,0.0,0.0
9,2018-11-09,3,None,10,11,False,"{'meta': None, 'task': 10, 'user': 11, 'level'...",notifications.notification,927,11,11,0.0,0.0


# Análise
Primeiro eu vou explorar um pouco sobre a distribuição na quantiadde de notificações por usuário.

In [192]:
students_not_count_series = notifications_with_user_grades_df.groupby("pk_user").size()

Com o grupo  de usuários que mais recebeu notificação e o grupo de usuários que menos recebeu notificações, eu vou tentar analisar qual a diferença disso e o rendimento comparado entre eles, irei colocar o 25% de cada. 

Existe um "lixo" nos dados que são os estudantes criados pelos adm, geralmente seu nomes são compostos por "Estudante X"

In [193]:
students_not_count_series.sort_values(ascending=False)

pk_user
40    617
34    456
29    433
56    422
68    387
11    345
78    338
30    333
50    316
17    306
72    299
41    286
52    269
43    262
63    243
69    234
46    221
74    215
28    213
84    202
15    183
16    172
45    168
59    153
75    153
76    144
51    142
53    140
13    140
19    133
     ... 
83     79
85     79
14     73
44     72
70     71
54     71
27     69
20     67
55     65
57     64
64     62
26     62
25     58
60     57
39     56
47     55
35     55
77     52
37     51
12     47
61     41
62     38
36     38
33     37
42     35
48     32
81     31
86     30
88     26
21     14
Length: 78, dtype: int64

In [194]:
students_not_count_series.quantile([.1,.25, .5, .75, .9])

0.10     38.00
0.25     62.50
0.50    104.00
0.75    197.25
0.90    321.10
dtype: float64

Como da pra ver a distância é bem grande entre o 1ª quartil (.25) e o menores 10% assim como a distância é ainda mais grotesca com entre o 3ª quartil e o top 10%, quase o dobro de notificações recebidas.

## distribuição por nível de pendência
Os níveis de pendência variam entre 4 valores, sendo que o valor 1 e 2 representam o mesmo dado,
1. O valor 1 e 2 representam tarefas que ainda estão no prazo
2. O valor 3 representa tarefas que estão atrasadas
3. O valor 4 representa tarefas que não podem mais ser completadas/perdidas

In [195]:
notifications_with_user_grades_df.loc[notifications_with_user_grades_df["pk_user"] == 40, ["v1", "v2"]].head(1)

,v1,v2
4030,0.0,0.0


In [196]:
count_notification_level_by_user = notifications_with_user_grades_df.groupby(["pk_user", "level"]).size()

In [197]:
count_notification_level_by_user.reset_index().pivot(index="pk_user", columns="level", values=0)

level,1,2,3,4
pk_user,,,,
11,71.0,NaN,179.0,95.0
12,45.0,NaN,2.0,NaN
13,58.0,NaN,39.0,43.0
14,23.0,NaN,7.0,43.0
15,64.0,NaN,76.0,43.0
16,71.0,NaN,58.0,43.0
17,73.0,NaN,190.0,43.0
18,54.0,NaN,30.0,NaN
19,60.0,NaN,73.0,NaN


# Indexando as notificações por tempo, análise temporal

In [198]:
notifications_with_user_grades_df.columns

Index(['creation_date', 'level', 'meta', 'task', 'user', 'viewed', 'fields',
       'model', 'pk_notifications', 'pk_user', 'id_estudante', 'v1', 'v2'],
      dtype='object')

In [199]:
notifications_with_user_grades_df.index = notifications_with_user_grades_df["creation_date"]

In [200]:
notifications_with_user_grades_df.index

DatetimeIndex(['2018-11-05', '2018-11-06', '2018-11-06', '2018-11-06',
               '2018-11-07', '2018-11-07', '2018-11-08', '2018-11-08',
               '2018-11-08', '2018-11-09',
               ...
               '2019-01-04', '2019-01-13', '2019-01-14', '2019-01-16',
               '2019-01-17', '2019-01-18', '2019-01-19', '2019-01-20',
               '2019-01-21', '2019-01-22'],
              dtype='datetime64[ns]', name='creation_date', length=11404, freq=None)

In [208]:
notifications_with_user_grades_df.resample('M').agg({"v1": "mean", "v2": "mean"})

,v1,v2
creation_date,,
2018-11-30,4.025120,4.690423
2018-12-31,3.089561,3.603879
2019-01-31,4.214038,4.814518


Dado muito bom: a média de notificações visualizadas e quantidade de notificações visualizadas por semana de cada usuário

In [221]:
notifications_with_user_grades_df.groupby(["pk_user", pd.Grouper(freq="W")]).agg({"viewed": ["mean", "sum"]}).reset_index()

pk_user creation_date    viewed      
                               mean   sum
0        11    2018-11-11  0.000000   0.0
1        11    2018-11-18  0.000000   0.0
2        11    2018-11-25  0.147541   9.0
3        11    2018-12-02  0.000000   0.0
4        11    2018-12-09  0.157895   9.0
5        11    2018-12-16  0.000000   0.0
6        11    2018-12-23  0.000000   0.0
7        11    2019-01-06  0.000000   0.0
8        11    2019-01-13  0.000000   0.0
9        11    2019-01-20  0.000000   0.0
10       11    2019-01-27  0.000000   0.0
11       12    2018-11-11  0.625000   5.0
12       12    2018-11-18  0.714286   5.0
13       12    2018-11-25  0.142857   1.0
14       12    2018-12-02  0.200000   1.0
15       12    2018-12-16  0.000000   0.0
16       12    2018-12-23  1.000000   1.0
17       12    2019-01-20  0.100000   1.0
18       12    2019-01-27  0.400000   2.0
19       13    2018-11-11  0.444444   4.0
20       13    2018-11-18  0.562500   9.0
21       13    2018-11-25  0.083333   2.0
22       13    2018-12-02  0.545455   6.0
23       13    2018-12-09  0.142857   1.0
24       13    2018-12-16  0.000000   0.0
25       13    2018-12-23  0.000000   0.0
26       13    2019-01-06  0.000000   0.0
27       13    2019-01-13  0.000000   0.0
28       13    2019-01-20  0.000000   0.0
29       13    2019-01-27  0.500000   7.0
..      ...           ...       ...   ...
733      85    2018-12-02  0.285714   2.0
734      85    2018-12-09  0.428571   3.0
735      85    2018-12-16  0.000000   0.0
736      85    2018-12-23  1.000000   2.0
737      85    2019-01-06  0.000000   0.0
738      85    2019-01-13  0.000000   0.0
739      85    2019-01-20  0.440000  11.0
740      85    2019-01-27  0.000000   0.0
741      86    2018-11-11  0.100000   1.0
742      86    2018-11-18  0.625000   5.0
743      86    2018-12-16  1.000000   1.0
744      86    2019-01-13  0.000000   0.0
745      86    2019-01-20  0.625000   5.0
746      86    2019-01-27  0.000000   0.0
747      87    2018-11-11  0.357143   5.0
748      87    2018-11-18  0.687500  11.0
749      87    2018-11-25  0.043478   1.0
750      87    2018-12-02  0.266667   4.0
751      87    2018-12-16  0.500000   1.0
752      87    2019-01-13  0.000000   0.0
753      87    2019-01-20  0.000000   0.0
754      87    2019-01-27  0.500000   5.0
755      88    2018-11-11  0.875000   7.0
756      88    2018-11-18  0.333333   1.0
757      88    2018-12-16  0.000000   0.0
758      88    2018-12-23  0.000000   0.0
759      88    2019-01-06  0.000000   0.0
760      88    2019-01-13  0.000000   0.0
761      88    2019-01-20  0.333333   2.0
762      88    2019-01-27  0.000000   0.0

[763 rows x 4 columns]